In [1]:
using Rocket
using BenchmarkTools

In [2]:
plusone(d) = one(d) / (one(d) + d)

function loop_draft()
    m_ab = lazy(Float64)
    m_ac = lazy(Float64)

    m_ba = lazy(Float64)
    m_bc = lazy(Float64)

    m_ca = lazy(Float64)
    m_cb = lazy(Float64)
    
    m_cy = lazy(Float64)
    m_yc = Subject(Float64)
    
    q1 = BehaviorSubject(0.0)
    q2 = BehaviorSubject(0.0)
    q3 = BehaviorSubject(0.0)
    q4 = BehaviorSubject(0.0)
    
    q_algorithm = discontinue() + map(Float64, (d) -> 1.0 / sum(d))#  + share()

    q_up_1 = combineLatest(m_ac, m_ca) |> q_algorithm
    q_up_2 = combineLatest(m_bc, m_cb) |> q_algorithm
    q_up_3 = combineLatest(m_ab, m_ba) |> q_algorithm
    q_up_4 = combineLatest(m_cy, m_yc) |> q_algorithm

    subscribe!(q_up_1, q1)
    subscribe!(q_up_2, q2)
    subscribe!(q_up_3, q3)
    subscribe!(q_up_4, q4)
    
#     s1 = subscribe!(q1, logger("q1"))
#     s2 = subscribe!(q2, logger("q2"))
#     s3 = subscribe!(q3, logger("q3"))
    
    m_algorithm = map(Float64, plusone)
    
    set!(m_ab, q1 |> m_algorithm)
    set!(m_ba, q2 |> m_algorithm)
    set!(m_bc, q3 |> m_algorithm)
    set!(m_ba, q2 |> m_algorithm)
#     set!(m_ca, q2 |> m_algorithm)
#     set!(m_cb, q1 |> m_algorithm)
    set!(m_ca, combineLatest(q2, q4) |> map(Float64, sum))
    set!(m_cb, combineLatest(q1, q4) |> map(Float64, sum))
    set!(m_cy, combineLatest(q1, q2) |> map(Float64, sum))
    
#     sa1 = subscribe!(m_ab |> skip_complete(), logger("m_ab"))
#     sa2 = subscribe!(m_ba |> skip_complete(), logger("m_ba"))
#     sb1 = subscribe!(m_bc |> skip_complete(), logger("m_bc"))
#     sb2 = subscribe!(m_ba |> skip_complete(), logger("m_ba"))
#     sc1 = subscribe!(m_ca |> skip_complete(), logger("m_ca"))
#     sc2 = subscribe!(m_cb |> skip_complete(), logger("m_cb"))
    
#     next!(q1, q1.props.current + 1)
#     next!(q2, q2.props.current + 1)
#     next!(q3, q3.props.current + 1)
    # next!(m_yc, 1.0);
    # next!(m_yc, 1.0);

    for i in 1:10
        next!(m_yc, 1.0);
    end
    
    return (q1.props.current, q2.props.current, q3.props.current, q4.props.current)
end

loop_draft (generic function with 1 method)

In [5]:
@btime loop_draft()

  19.965 μs (408 allocations: 15.89 KiB)


(0.0, 0.8793858039394876, 0.6527037124959668, 0.5320882571447347)

In [4]:
@time loop_draft()

  0.000096 seconds (408 allocations: 15.891 KiB)


(0.0, 0.8793858039394876, 0.6527037124959668, 0.5320882571447347)